# Import Data

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import math

In [ ]:
df = pd.read_csv("Ming1Jul67_total_order.csv")
df.head(5)

In [ ]:
print(df.info())

In [ ]:
start_date = df["order_completed_at"].head(1).iloc[0]
end_date   = df["order_completed_at"].tail(1).iloc[0]
print(f"{start_date} to {end_date}")

In [ ]:
df['order_completed_at'] = pd.to_datetime(df['order_completed_at'])

# Data Processing

### Split 80 Percen %

In [ ]:
# Setting 80 percent data for training
training_data_len = math.ceil(len(df) * .8)
training_data_len 
 
#Splitting the dataset
train_data = df[:training_data_len].iloc[:,1:2] 
test_data = df[training_data_len:].iloc[:,1:2]
print(train_data.shape, test_data.shape)

### Split By Date

In [ ]:
import pandas as pd

# สมมติว่า df เป็น DataFrame ที่มีคอลัมน์ 'Date' เป็น index
# และมีข้อมูลอื่นๆ ในคอลัมน์ถัดไป

# กำหนดวันที่ตัด
cut_off_date = '2024-05-01'

# แบ่งข้อมูล
train_data = df[df.order_completed_at < cut_off_date].iloc[:, 1:2]
test_data = df[df.order_completed_at >= cut_off_date].iloc[:, 1:2]

print(train_data.shape, test_data.shape)

# ถ้าต้องการดูช่วงวันที่ของแต่ละชุดข้อมูล
print("Train data range:", train_data.index.min(), "to", train_data.index.max())
print("Test data range:", test_data.index.min(), "to", test_data.index.max())

In [ ]:
train_data

In [ ]:
# Selecting Open Price values
dataset_train = train_data.unique_order_count.values 
# Reshaping 1D to 2D array
dataset_train = np.reshape(dataset_train, (-1,1)) 
dataset_train.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
scaled_train = scaler.fit_transform(dataset_train)
 
print(scaled_train[:5])

In [ ]:
# Selecting Open Price values
dataset_test = test_data.unique_order_count.values 
# Reshaping 1D to 2D array
dataset_test = np.reshape(dataset_test, (-1,1))  
# Normalizing values between 0 and 1
scaled_test = scaler.fit_transform(dataset_test)  
print(*scaled_test[:5])

In [ ]:
X_train = []
y_train = []
for i in range(50, len(scaled_train)):
    X_train.append(scaled_train[i-50:i, 0])
    y_train.append(scaled_train[i, 0])
    if i <= 51:
        print(X_train)
        print(y_train)
        print()

In [ ]:
X_test = []
y_test = []
for i in range(50, len(scaled_test)):
    X_test.append(scaled_test[i-50:i, 0])
    y_test.append(scaled_test[i, 0])

In [ ]:
# The data is converted to Numpy array
X_train, y_train = np.array(X_train), np.array(y_train)
 
#Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
y_train = np.reshape(y_train, (y_train.shape[0],1))
print("X_train :",X_train.shape,"y_train :",y_train.shape)

In [ ]:
# The data is converted to numpy array
X_test, y_test = np.array(X_test), np.array(y_test)
 
#Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))
print("X_test :",X_test.shape,"y_test :",y_test.shape)

In [ ]:
print(X_test[0])

In [ ]:
# importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import GRU, Bidirectional
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import mean_squared_error

# Modeling 

In [ ]:
# initializing the RNN
regressor = Sequential()
 
# adding RNN layers and dropout regularization
regressor.add(SimpleRNN(units = 50, 
                        activation = "tanh",
                        return_sequences = True,
                        input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))
 
regressor.add(SimpleRNN(units = 50, 
                        activation = "tanh",
                        return_sequences = True))
 
regressor.add(SimpleRNN(units = 50,
                        activation = "tanh",
                        return_sequences = True))
 
regressor.add( SimpleRNN(units = 50))
 
# adding the output layer
regressor.add(Dense(units = 1,activation='sigmoid'))
 
# compiling RNN
regressor.compile(optimizer = SGD(learning_rate=0.01,
                                  decay=1e-6, 
                                  momentum=0.9, 
                                  nesterov=True), 
                  loss = "mean_squared_error")
 
# fitting the model
regressor.fit(X_train, y_train, epochs = 20, batch_size = 2)
regressor.summary()

In [ ]:
#Initialising the model
regressorLSTM = Sequential()
 
#Adding LSTM layers
regressorLSTM.add(LSTM(50, 
                       return_sequences = True, 
                       input_shape = (X_train.shape[1],1)))
regressorLSTM.add(LSTM(50, 
                       return_sequences = False))
regressorLSTM.add(Dense(25))
 
#Adding the output layer
regressorLSTM.add(Dense(1))
 
#Compiling the model
regressorLSTM.compile(optimizer = 'adam',
                      loss = 'mean_squared_error',
                      metrics = ["accuracy"])
 
#Fitting the model
regressorLSTM.fit(X_train, 
                  y_train, 
                  batch_size = 1, 
                  epochs = 12)
regressorLSTM.summary()

In [ ]:
#Initialising the model
regressorGRU = Sequential()
 
# GRU layers with Dropout regularisation
regressorGRU.add(GRU(units=50, 
                     return_sequences=True,
                     input_shape=(X_train.shape[1],1),
                     activation='tanh'))
regressorGRU.add(Dropout(0.2))
 
regressorGRU.add(GRU(units=50, 
                     return_sequences=True,
                     activation='tanh'))
 
regressorGRU.add(GRU(units=50, 
                     return_sequences=True,
                     activation='tanh'))
 
regressorGRU.add(GRU(units=50, 
                     activation='tanh'))
 
# The output layer
regressorGRU.add(Dense(units=1,
                       activation='relu'))
# Compiling the RNN
regressorGRU.compile(optimizer=SGD(learning_rate=0.01, 
                                   decay=1e-7, 
                                   momentum=0.9, 
                                   nesterov=False),
                     loss='mean_squared_error')
 
# Fitting the data
regressorGRU.fit(X_train,y_train,epochs=20,batch_size=1)
regressorGRU.summary()

In [ ]:
# predictions with X_test data
y_RNN = regressor.predict(X_test)
y_LSTM = regressorLSTM.predict(X_test)
y_GRU = regressorGRU.predict(X_test)

In [ ]:
# scaling back from 0-1 to original
y_RNN_O = scaler.inverse_transform(y_RNN) 
y_LSTM_O = scaler.inverse_transform(y_LSTM) 
y_GRU_O = scaler.inverse_transform(y_GRU)

In [ ]:
fig, axs = plt.subplots(3,figsize =(18,12),sharex=True, sharey=True)
fig.suptitle('Model Predictions')
 
#Plot for RNN predictions
axs[0].plot(train_data.index[150:], train_data.unique_order_count[150:], label = "train_data", color = "b")
axs[0].plot(test_data.index, test_data.unique_order_count, label = "test_data", color = "g")
axs[0].plot(test_data.index[50:], y_RNN_O, label = "y_RNN", color = "brown")
axs[0].legend()
axs[0].title.set_text("Basic RNN")
 
#Plot for LSTM predictions
axs[1].plot(train_data.index[150:], train_data.unique_order_count[150:], label = "train_data", color = "b")
axs[1].plot(test_data.index, test_data.unique_order_count, label = "test_data", color = "g")
axs[1].plot(test_data.index[50:], y_LSTM_O, label = "y_LSTM", color = "orange")
axs[1].legend()
axs[1].title.set_text("LSTM")
 
#Plot for GRU predictions
axs[2].plot(train_data.index[150:], train_data.unique_order_count[150:], label = "train_data", color = "b")
axs[2].plot(test_data.index, test_data.unique_order_count, label = "test_data", color = "g")
axs[2].plot(test_data.index[50:], y_GRU_O, label = "y_GRU", color = "red")
axs[2].legend()
axs[2].title.set_text("GRU")
 
plt.xlabel("Days")
plt.ylabel("Open price")
 
plt.show()

In [ ]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error)
from math import sqrt


pred_dict = {"RNN":y_RNN_O,"LSTM":y_LSTM_O, "GRU":y_GRU_O}
y_true = test_data[50:].unique_order_count
# /  

for name, pred in pred_dict.items():
    mae = mean_absolute_error(y_true, pred)
    mse = mean_squared_error(y_true,pred)
    rmse = math.sqrt(mse)
    mape =mean_absolute_percentage_error(y_true,pred)
    print(f"==== {name} ====")
    print(f"MAE: {mae}")
    print(f"MSE {mse}")
    print(f"RMSE {rmse}")
    print(f"MAPE: {mape}\n")
# print(RNN_mae)

In [ ]:
model = regressor.save('train_regressorRNN_1Jul67.keras')
model = regressorLSTM.save('train_regressorLSTM_1Jul67.keras')
model = regressorGRU.save('train_regressorGRU_1Jul67.keras')